# Data Collection, Cleaning, and Concatenation

This notebook collects, joins, and exports all of the initial variables used in the feature set

Imports

In [1]:
import os
from dotenv import load_dotenv
import numpy as np
import pandas as pd
import finnhub
import yfinance as yf
import talib as ta

Get dates for a ten-year window and convert to UNIX Timestamp integers

In [2]:
# Create datetime objects
start = pd.Timestamp("2011-08-01 23:59:00")
end = pd.Timestamp("2022-08-01 23:59:00")

# Convert datetime objects to UNIX timestamp integers
start_unix = int(pd.Timestamp.timestamp(start))
end_unix = int(pd.Timestamp.timestamp(end))

Get SPY data from FinnHub

In [3]:
# Connect to FinnHub API
load_dotenv()
finnhub_api_key = os.getenv('FINNHUB_API_KEY')
fh = finnhub.Client(api_key=finnhub_api_key)

# Note: price data is already adjusted by Finnhub
spy_ohlcv = fh.stock_candles('SPY', 'D', start_unix, end_unix)
spy_ohlcv = pd.DataFrame(spy_ohlcv)
spy_ohlcv.tail()

,c,h,l,o,s,t,v
2764,390.89,394.06,389.95,393.84,ok,1658793600,52946393
2765,401.04,402.88,394.05,394.36,ok,1658880000,82342106
2766,406.07,406.80,398.15,401.89,ok,1658966400,73966563
2767,411.99,413.03,406.77,407.58,ok,1659052800,87003672
2768,410.77,413.41,408.40,409.15,ok,1659312000,69997471


Check for nulls or missing data

In [4]:
# The `s` column indicates the response status
# Finnhub returns the string "no_data" for missing rows
spy_ohlcv.loc[spy_ohlcv['s'] == 'no_data']

,c,h,l,o,s,t,v


In [5]:
# Check for any null rows
spy_ohlcv.isnull().values.any()

False

Convert the returned UNIX timestamps to date objects and index the date column

In [6]:
spy_ohlcv['date'] = pd.to_datetime(spy_ohlcv['t'], unit='s')
spy_ohlcv = spy_ohlcv.set_index(spy_ohlcv['date'])
spy_ohlcv.tail()

,c,h,l,o,s,t,v,date
date,,,,,,,,
2022-07-26,390.89,394.06,389.95,393.84,ok,1658793600,52946393,2022-07-26
2022-07-27,401.04,402.88,394.05,394.36,ok,1658880000,82342106,2022-07-27
2022-07-28,406.07,406.80,398.15,401.89,ok,1658966400,73966563,2022-07-28
2022-07-29,411.99,413.03,406.77,407.58,ok,1659052800,87003672,2022-07-29
2022-08-01,410.77,413.41,408.40,409.15,ok,1659312000,69997471,2022-08-01


Drop columns and create the features DataFrame 'X'

In [22]:
X = spy_ohlcv.drop(columns=['s', 't', 'date'])
X.columns = ['close', 'high', 'low', 'open', 'volume']

Check dates

In [23]:
display(X.shape, X.head(), X.tail())

(2769, 5)

,close,high,low,open,volume
date,,,,,
2011-08-01,128.78,130.96,127.53,130.84,325790824
2011-08-02,125.49,128.50,125.49,127.81,346653757
2011-08-03,126.17,126.31,123.53,125.66,371029415
2011-08-04,120.26,124.62,120.06,124.42,520721783
2011-08-05,120.08,122.07,116.86,121.76,655561584


,close,high,low,open,volume
date,,,,,
2022-07-26,390.89,394.06,389.95,393.84,52946393
2022-07-27,401.04,402.88,394.05,394.36,82342106
2022-07-28,406.07,406.80,398.15,401.89,73966563
2022-07-29,411.99,413.03,406.77,407.58,87003672
2022-08-01,410.77,413.41,408.40,409.15,69997471


## Create Variables with TA Lib

In [24]:
close = X['close']
high = X['high']
low = X['low']

# Hilbert Transform - Instantaneous Trendline
X['trend'] = ta.HT_TRENDLINE(close)

# RSI
X['rsi'] = ta.RSI(close, timeperiod=20)

# Stochastic RSI
X['rsi_fast_k'], X['rsi_fast_d'] = ta.STOCHRSI(
    close, 
    timeperiod=14, 
    fastk_period=5, 
    fastd_period=3, 
    fastd_matype=0
)

# Williams' %R
X['williams_r'] = ta.WILLR(high, low, close, timeperiod=14)

# MACD, MACD Signal, MACD Histogram
X['macd'], X['macd_signal'], X['macd_hist'] = ta.MACD(
    close, 
    fastperiod=12, 
    slowperiod=26, 
    signalperiod=9
)

# Moving Averages
X['ma_20'] = ta.MA(close, timeperiod=20, matype=0)
X['ma_50'] = ta.MA(close, timeperiod=50, matype=0)
X['ma_65'] = ta.MA(close, timeperiod=65, matype=0)
X['ma_200'] = ta.MA(close, timeperiod=200, matype=0)

# Bollinger Bands
X['bb_upp'], X['bb_mid'], X['bb_low'] = ta.BBANDS(
    close, 
    timeperiod=5, 
    nbdevup=2, 
    nbdevdn=2, 
    matype=0
)

# Percentage Price Oscillator
X['ppo'] = ta.PPO(close, fastperiod=12, slowperiod=26, matype=0)

# Momentum
X['mom'] = ta.MOM(close, timeperiod=10)

# Rate of Change
X['roc'] = ta.ROC(close, timeperiod=10)

# Exponential Moving Averages
X['ema_20'] = ta.EMA(close, timeperiod=20)
X['ema_50'] = ta.EMA(close, timeperiod=50)
X['ema_65'] = ta.EMA(close, timeperiod=65)
X['ema_200'] = ta.EMA(close, timeperiod=200)

Slice the data from Aug 1, 2012 onward

In [25]:
# Remove all data before Aug 1, 2012 for an exact ten years
X = X.loc['2012-08-01':]
display(X.shape, X.head())

(2516, 27)

,close,high,low,open,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,...,bb_upp,bb_mid,bb_low,ppo,mom,roc,ema_20,ema_50,ema_65,ema_200
date,,,,,,,,,,,,,,,,,,,,,
2012-08-01,137.590,138.73,137.4000,138.70,138293740,136.387994,55.587587,36.624310,70.086458,-27.722301,...,139.607390,137.7650,135.922610,0.693206,0.220,0.160151,136.139105,135.287401,135.135725,130.535144
2012-08-02,136.640,137.57,135.5800,136.55,199556580,136.561699,52.970383,0.000000,36.753125,-42.780155,...,139.387508,137.8590,136.330492,0.577563,-1.090,-0.791403,136.186809,135.340444,135.181310,130.595888
2012-08-03,139.349,139.64,136.6794,138.56,157824975,136.708467,58.793873,97.029563,44.551291,-4.402421,...,139.866511,137.9938,136.121089,0.523204,2.879,2.109621,136.487970,135.497642,135.307603,130.682984
2012-08-06,139.620,140.17,139.5600,139.72,86327738,136.843608,59.324250,100.000000,65.676521,-7.703081,...,140.441135,138.1818,135.922465,0.445066,4.530,3.353320,136.786259,135.659303,135.438282,130.771910
2012-08-07,140.320,140.92,140.0300,140.18,109545089,137.054508,60.699634,100.000000,99.009854,-7.604563,...,141.441336,138.7038,135.966264,0.560776,6.395,4.775061,137.122805,135.842076,135.586213,130.866915


Check for nulls

In [37]:
X[X.isnull().any(axis=1)]

,close,high,low,open,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,...,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
date,,,,,,,,,,,,,,,,,,,,,


## Get Bond Data from CSVs

## Get features from Yahoo!

In [27]:
# Row 1: USD/x Currency pairs
# Row 2: US Indices
# Row 3: Oil, Gold
yf_tickers = \
"CAD=X CNY=X INR=X SGD=X HKD=X AUD=X JPY=X EUR=X \
^GSPC ^NDX ^DJI ^DJT ^RUT \
CL=F GC=F"

# Fetch market data and use only the adjusted close
markets_ohlcv = yf.download(yf_tickers, start='2012-08-01', end='2022-08-02')
markets = markets_ohlcv['Adj Close'].drop(index='2022-08-02')
markets.tail()

[*********************100%***********************]  15 of 15 completed


,AUD=X,CAD=X,CL=F,CNY=X,EUR=X,GC=F,HKD=X,INR=X,JPY=X,SGD=X,^DJI,^DJT,^GSPC,^NDX,^RUT
Date,,,,,,,,,,,,,,,
2022-07-26,1.438600,1.28507,94.980003,6.7495,0.97800,1717.699951,7.84913,79.722504,136.431000,1.38603,31761.539062,13614.139648,3921.050049,12086.900391,1805.250000
2022-07-27,1.439390,1.28737,97.260002,6.7623,0.98721,1719.099976,7.84926,79.841202,136.970001,1.38900,32197.589844,13847.009766,4023.610107,12601.469727,1848.339966
2022-07-28,1.429613,1.28171,96.419998,6.7574,0.97950,1750.300049,7.84910,79.847397,136.110992,1.38170,32529.630859,14275.299805,4072.429932,12717.870117,1873.030029
2022-07-29,1.428400,1.28070,98.620003,6.7458,0.98113,1762.900024,7.84950,79.571503,134.397003,1.37910,32845.128906,14609.000000,4130.290039,12947.969727,1885.229980
2022-08-01,1.434470,1.28156,93.889999,6.7432,0.97960,1769.000000,7.84944,79.194801,133.406998,1.38150,32798.398438,14634.089844,4118.629883,12940.780273,1883.310059


Format column Names

In [28]:
markets.columns = [
    'usd_aud', 'usd_cad', 'crude_oil', 'usd_cny', 
    'usd_eur', 'gold', 'usd_hkd', 'usd_inr', 'usd_jpy', 'usd_sgd', 
    'dji', 'djt', 'gspc', 'ndx', 'rut'
]

markets = markets.reindex(columns=[
    'gspc', 'ndx', 'rut', 'dji', 'djt', 'crude_oil', 'gold',
    'usd_aud', 'usd_cad', 'usd_cny', 'usd_eur',
    'usd_hkd', 'usd_inr', 'usd_jpy', 'usd_sgd',
])

markets.head()

,gspc,ndx,rut,dji,djt,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
Date,,,,,,,,,,,,,,,
2012-08-01,1375.319946,2635.129883,771.130005,12976.129883,4986.709961,88.910004,1603.699951,0.95374,1.00320,6.3618,0.8134,7.75376,55.647999,78.120003,1.24572
2012-08-02,1365.000000,2625.520020,768.599976,12878.879883,4984.149902,87.129997,1587.400024,0.95511,1.00472,6.3688,0.8169,7.75500,55.467999,78.379997,1.24740
2012-08-03,1390.989990,2676.000000,788.479980,13096.169922,5086.310059,91.400002,1606.000000,0.95613,1.00705,6.3676,0.8210,7.75426,55.838001,78.220001,1.24870
2012-08-06,1394.229980,2694.090088,794.349976,13117.509766,5082.359863,92.199997,1612.900024,0.94706,1.00057,6.3715,0.8046,7.75430,55.748001,78.610001,1.24060
2012-08-07,1401.349976,2717.159912,801.340027,13168.599609,5092.459961,93.669998,1609.699951,0.94670,1.00060,6.3740,0.8071,7.75500,55.518002,78.199997,1.24150


Check for nulls

In [29]:
markets[markets.isna().any(axis=1)]

,gspc,ndx,rut,dji,djt,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
Date,,,,,,,,,,,,,,,
2012-09-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.972940,0.98600,6.3486,0.79510,7.75590,55.518002,78.290001,1.24890
2012-10-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.965900,0.99920,6.2624,0.77375,7.75020,53.548000,79.650002,1.22140
2012-10-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.967140,1.00070,6.2440,0.77470,7.75000,54.077999,79.764000,1.22120
2012-11-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.963110,0.99573,6.2314,0.77730,7.75058,55.108002,82.573997,1.22437
2012-12-04,1407.050049,2667.889893,822.119995,12951.780273,5074.339844,88.500000,1694.400024,NaN,NaN,6.2286,NaN,NaN,54.768002,82.133003,1.21840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.394370,1.27566,6.3242,0.88361,7.79974,74.682404,114.960999,1.34616
2022-04-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.349500,1.26030,6.3774,0.92413,7.84262,76.308800,126.179001,1.35624
2022-05-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.396100,1.27237,6.6976,0.93184,7.84906,77.692001,127.285004,1.36790


#### *We have null values!*

Since we have existing data alongside of nulls, we want to fill the gaps rather than delete existing data

By chaining `df.ffill()` and `df.bfill()` we get the best of both forward-filling and back-filling

In [30]:
markets = markets.ffill().bfill()

Check shape and dates

In [31]:
display(markets.shape, markets.head(), markets.tail())

(2609, 15)

,gspc,ndx,rut,dji,djt,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
Date,,,,,,,,,,,,,,,
2012-08-01,1375.319946,2635.129883,771.130005,12976.129883,4986.709961,88.910004,1603.699951,0.95374,1.00320,6.3618,0.8134,7.75376,55.647999,78.120003,1.24572
2012-08-02,1365.000000,2625.520020,768.599976,12878.879883,4984.149902,87.129997,1587.400024,0.95511,1.00472,6.3688,0.8169,7.75500,55.467999,78.379997,1.24740
2012-08-03,1390.989990,2676.000000,788.479980,13096.169922,5086.310059,91.400002,1606.000000,0.95613,1.00705,6.3676,0.8210,7.75426,55.838001,78.220001,1.24870
2012-08-06,1394.229980,2694.090088,794.349976,13117.509766,5082.359863,92.199997,1612.900024,0.94706,1.00057,6.3715,0.8046,7.75430,55.748001,78.610001,1.24060
2012-08-07,1401.349976,2717.159912,801.340027,13168.599609,5092.459961,93.669998,1609.699951,0.94670,1.00060,6.3740,0.8071,7.75500,55.518002,78.199997,1.24150


,gspc,ndx,rut,dji,djt,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
Date,,,,,,,,,,,,,,,
2022-07-26,3921.050049,12086.900391,1805.250000,31761.539062,13614.139648,94.980003,1717.699951,1.438600,1.28507,6.7495,0.97800,7.84913,79.722504,136.431000,1.38603
2022-07-27,4023.610107,12601.469727,1848.339966,32197.589844,13847.009766,97.260002,1719.099976,1.439390,1.28737,6.7623,0.98721,7.84926,79.841202,136.970001,1.38900
2022-07-28,4072.429932,12717.870117,1873.030029,32529.630859,14275.299805,96.419998,1750.300049,1.429613,1.28171,6.7574,0.97950,7.84910,79.847397,136.110992,1.38170
2022-07-29,4130.290039,12947.969727,1885.229980,32845.128906,14609.000000,98.620003,1762.900024,1.428400,1.28070,6.7458,0.98113,7.84950,79.571503,134.397003,1.37910
2022-08-01,4118.629883,12940.780273,1883.310059,32798.398438,14634.089844,93.889999,1769.000000,1.434470,1.28156,6.7432,0.97960,7.84944,79.194801,133.406998,1.38150


In [32]:
X = X.join(markets, on=X.index)

### DataFrame Overview

Check X for null values

In [38]:
X[X.isnull().any(axis=1)]

,close,high,low,open,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,...,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
date,,,,,,,,,,,,,,,,,,,,,


#### *More null values!*

Again, since we have data existing alongside null values, we'll fill the nulls so as not to delete good data

In [34]:
X = X.ffill().bfill()
X[X.isnull().any(axis=1)]

,close,high,low,open,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,...,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
date,,,,,,,,,,,,,,,,,,,,,


#### *No more nulls!*

We can sleep well now... 😴

In [35]:
print(X.shape)
display(X.head())
display(X.tail())

(2516, 42)


,close,high,low,open,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,...,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
date,,,,,,,,,,,,,,,,,,,,,
2012-08-01,137.590,138.73,137.4000,138.70,138293740,136.387994,55.587587,36.624310,70.086458,-27.722301,...,88.910004,1603.699951,0.95374,1.00320,6.3618,0.8134,7.75376,55.647999,78.120003,1.24572
2012-08-02,136.640,137.57,135.5800,136.55,199556580,136.561699,52.970383,0.000000,36.753125,-42.780155,...,87.129997,1587.400024,0.95511,1.00472,6.3688,0.8169,7.75500,55.467999,78.379997,1.24740
2012-08-03,139.349,139.64,136.6794,138.56,157824975,136.708467,58.793873,97.029563,44.551291,-4.402421,...,91.400002,1606.000000,0.95613,1.00705,6.3676,0.8210,7.75426,55.838001,78.220001,1.24870
2012-08-06,139.620,140.17,139.5600,139.72,86327738,136.843608,59.324250,100.000000,65.676521,-7.703081,...,92.199997,1612.900024,0.94706,1.00057,6.3715,0.8046,7.75430,55.748001,78.610001,1.24060
2012-08-07,140.320,140.92,140.0300,140.18,109545089,137.054508,60.699634,100.000000,99.009854,-7.604563,...,93.669998,1609.699951,0.94670,1.00060,6.3740,0.8071,7.75500,55.518002,78.199997,1.24150


,close,high,low,open,volume,trend,rsi,rsi_fast_k,rsi_fast_d,williams_r,...,crude_oil,gold,usd_aud,usd_cad,usd_cny,usd_eur,usd_hkd,usd_inr,usd_jpy,usd_sgd
date,,,,,,,,,,,,,,,,,,,,,
2022-07-26,390.89,394.06,389.95,393.84,52946393,385.898000,49.785606,0.000000,34.261989,-31.880577,...,94.980003,1717.699951,1.438600,1.28507,6.7495,0.97800,7.84913,79.722504,136.431000,1.38603
2022-07-27,401.04,402.88,394.05,394.36,82342106,386.804389,55.163433,98.608208,42.891288,-5.778894,...,97.260002,1719.099976,1.439390,1.28737,6.7623,0.98721,7.84926,79.841202,136.970001,1.38900
2022-07-28,406.07,406.80,398.15,401.89,73966563,387.952778,57.535789,100.000000,66.202736,-2.041387,...,96.419998,1750.300049,1.429613,1.28171,6.7574,0.97950,7.84910,79.847397,136.110992,1.38170
2022-07-29,411.99,413.03,406.77,407.58,87003672,389.536980,60.148111,100.000000,99.536069,-2.476780,...,98.620003,1762.900024,1.428400,1.28070,6.7458,0.98113,7.84950,79.571503,134.397003,1.37910
2022-08-01,410.77,413.41,408.40,409.15,69997471,391.093075,59.356006,90.899678,96.966559,-6.230824,...,93.889999,1769.000000,1.434470,1.28156,6.7432,0.97960,7.84944,79.194801,133.406998,1.38150
